In [2]:
import random
import numpy as np
from random import choice, randint

q = 37
d = q // 2 + 1

def check_inequality(n, k, r):
    half_k = k * (k + 1) // 2
    return (r == n) if half_k >= n else (r == half_k)

def make_weight1(n, q):
    slots = list(range(n))
    P = matrix(GF(q), n, n, 0)
    for i in range(n):
        index = choice(slots)
        slots.remove(index)
        P[i, index] = 1
    return P

def Schur_product(M, k1):
    Schur = []
    for i in range(k1):
        a_row = np.array(list(M[i]))
        for j in range(i, k1):
            b_row = np.array(list(M[j]))
            Schur.append(list(np.multiply(a_row, b_row)))
    return Schur

def generate_random_non_singular_matrix(q, k1):
    S = random_matrix(GF(q), k1, k1)
    while S.is_singular():
        S = random_matrix(GF(q), k1, k1)
    return S

def SchurCalc(M, q):
    k1 = M.rank()
    n = M.ncols()
    S = generate_random_non_singular_matrix(q, k1)
    P1 = make_weight1(n, q)
    G_cap_hidden1 = S * M * P1
    G_cap_Schur1 = Matrix(GF(q), Schur_product(G_cap_hidden1, k1))
    G1k = G_cap_Schur1.rank()
    return check_inequality(n, k1, G1k)

def codegen(q, d, a, b, c):
    """
    Строим AG-код по аффинной модели гиперэллиптической кривой:
        y^2 + c*y = x^3 + a*x + b.
    Преобразованием (Y = y + c/2) приводим её к форме Вейерштрасса:
        Y^2 = x^3 + a*x + (b + c^2/4).
    Вычисляем дискриминант и отбрасываем сингулярные случаи.
    """
    res = [None] * 5
    Fq = GF(q)
    A = Fq(a)
    # Приводим b + c^2/4 в поле GF(q)
    B = Fq(b) + Fq(c)**2 / Fq(4)
    disc = -16 * (4 * A**3 + 27 * B**2)
    if disc == 0:
        raise ValueError("Сингулярная кривая: дискриминант равен 0")
    
    # Аффинная модель: z = 1, уравнение: y^2 + c*y = x^3 + a*x + b
    R.<x, y> = PolynomialRing(Fq, 2)
    f = y**2 + c * y - (x**3 + a * x + b)
    C_affine = Curve(f)
    
    # Получаем функциональное поле и места порядка 1
    F_field = C_affine.function_field()
    places = F_field.places(1)
    print("a =", a, "b =", b, "c =", c, "disc =", disc, "число мест =", len(places))
    if len(places) <= 1:
        raise ValueError("Недостаточно мест для построения AG-кода")
    
    Q = places[0]
    del places[0]
    G = d * Q
    code = codes.EvaluationAGCode(places, G)
    res[0] = SchurCalc(code.generator_matrix(), q)
    
    try:
        code1 = codes.PuncturedCode(code, 1)
        res[1] = SchurCalc(code1.generator_matrix(), q)
    except ValueError:
        raise ValueError("Размерность кода не позволяет построить проколотый")
    
    try:
        code2 = code1.shortened([1])
        res[2] = SchurCalc(code2.generator_matrix(), q)
    except ValueError:
        raise ValueError("Размерность кода не позволяет построить укороченный")
    
    code3 = codes.SubfieldSubcode(code, Fq)
    res[3] = SchurCalc(code3.generator_matrix(), q)
    
    code41 = code.dual_code()
    code42 = codes.SubfieldSubcode(code41, Fq)
    code43 = code42.dual_code()
    res[4] = SchurCalc(code43.generator_matrix(), q)
    
    return res

# -------------------- ОСНОВНОЙ БЛОК --------------------
number_tests = 0   # Количество успешных тестов
probs = [0] * 5    # Средние значения (доли успеха) для пяти типов кодов
total_tests = 5  # Количество попыток

while number_tests < total_tests:
    a = randint(0, q - 1)
    b = randint(0, q - 1)
    c = randint(0, q - 1)
    try:
        res = codegen(q, d, a, b, c)
        if None not in res:
            for i in range(5):
                probs[i] = (probs[i] * number_tests + (1 if res[i] else 0)) / (number_tests + 1)
            number_tests += 1
    except Exception as e:
        print("error is", e)
        continue

code_names = ['AG-code', 'Punctured-code', 'Shortened-code', 'Subfield-code', 'Trace-code']
for i in range(5):
    print(f"{code_names[i]}: {probs[i]}")


a = 31 b = 34 c = 35 disc = 34 число мест = 39
a = 3 b = 5 c = 6 disc = 32 число мест = 30
a = 15 b = 1 c = 24 disc = 3 число мест = 40
a = 0 b = 8 c = 7 disc = 9 число мест = 48
a = 25 b = 32 c = 34 disc = 25 число мест = 49
AG-code: 1/5
Punctured-code: 3/5
Shortened-code: 2/5
Subfield-code: 1/5
Trace-code: 1/5
